## Sport challenges - Recommender System

#### Non avendo a disposizione un dataset di allenamenti con, per ognuno, un voto per ciascun utente, ed avendo invece un dataset molto sparso, con nessuna informazione in comune tra i vari utenti, ho optato per un approccio Content-based. In tale lavoro ci si baserà quindi sulla somiglianza tra le varie tipologie di allenamento effettuate dai vari utenti per arrivare quindi a stabilire relazioni di somiglianza tra gli interessi, in termini di allenamento, degli utenti stessi. A partire quindi dal dataset principale, si arriverà alla definizione di un Sistema di Raccomandazione che, a partire da un utente, restituirà la lista degli utenti a lui più vicini in termini di tipologie di allenamento.

In [244]:
!pip3 install -r requirements.txt

In [231]:
import pandas as pd
import numpy as np
import datetime
import haversine as hs
import datetime
import statistics
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

#### Leggo dal dataset ( https://sites.google.com/eng.ucsd.edu/fitrec-project/home ) i dati relativi a tutte le sessioni di allenamento. Ho optato per l'utilizzo del dataset endomondoHR_proper.json in quanto rappresenta un giusto compromesso tra la versione completa e quella semplificata / normalizzata.

In [7]:
data = []
with open('./endomondoHR_proper.json') as f:
    for l in f:
        data.append(eval(l))

#### Funzione che effettua il calcolo della distanza totale di un singolo allenamento (km) a partire dalla lista delle coordinate

In [8]:
def calculate_tranining_distance(single_training):
    total_distance = 0
    
    for index in range(len(single_training['latitude'])-2):
        lat1 = single_training['latitude'][index]
        lon1 = single_training['longitude'][index]
        lat2 = single_training['latitude'][index+1]
        lon2 = single_training['longitude'][index+1]

        loc1=(lat1,lon1)
        loc2=(lat2,lon2)
        total_distance = total_distance + hs.haversine(loc1,loc2)
        
    return int(total_distance)

#### Funzione che effettua il calcolo della durata totale del singolo allenamento (minuti) a partire dalla lista di timestam

In [9]:
def calculate_training_duration(single_training):
    start = single_training['timestamp'][0]
    finish = single_training['timestamp'][-1]
    
    start_date = datetime.datetime.fromtimestamp(start)
    finish_date = datetime.datetime.fromtimestamp(finish)
    
    difference = finish_date - start_date
    seconds_in_day = 24 * 60 * 60
    datetime.timedelta(0, 8, 562000)
    
    return int(divmod(difference.days * seconds_in_day + difference.seconds, 60)[0])

#### Funziona che effettua il calcolo della velocità media dell'allenamento (km/h) a partire dalla distanza totale e il tempo impiegato

In [10]:
def calculate_avg(distance, duration):
    if(duration > 0):
        return int((distance / (duration/60)))
    return 0

#### Funzione che effettua il calcolo della media dei battiti cardiaci a partire dalla lista contenente i campioni di battiti rilevati durante l'allenamento

In [11]:
def calculate_hear_rate_avg(single_training):
    return int(statistics.median(single_training['heart_rate']))

#### Funzione che effettua il calcolo del dislivello totale dell'allenamento a partire dalla lista contenente i campioni dell'altitude rilevata

In [12]:
def calculate_altitude_difference(single_training):
    total_altitude_difference = 0
    
    for index in range(len(single_training['altitude'])-2):
        if(single_training['altitude'][index + 1] >= single_training['altitude'][index]):
            total_altitude_difference = total_altitude_difference + (single_training['altitude'][index + 1] - single_training['altitude'][index])
        else:
            total_altitude_difference = total_altitude_difference + (single_training['altitude'][index] - single_training['altitude'][index + 1])
        
    return int(total_altitude_difference)

#### Funzione che trasforma la descrizione testuale del sesso in 1 (uomo) e 2 (donna)

In [13]:
def transform_sex_value(single_training):
    if(single_training['gender'] == 'male'):
        return 1
    else:
        return 2

#### Funzione che trasforma la descrizione testuale dello sport in 1 (bike) e 2 (run)

In [232]:
def transform_sport_value(single_training):
    if single_training['sport'] == 'bike' or single_training['sport'] == 'bike (transport)':
        return 1
    else:
        return 2

#### A partire dal dataset iniziale, viene creata una nuova lista contenente una versione rielaborata dei dati degli allenamenti (calcolo della distanza totale, calcolo della velocità media, calcolo della frequenza cardiaca media, calcolo del dislivello, calcolo della durata totale) in modo tale da consentirne una migliore fruzione ed analisi delle informazioni contenute.

In [15]:
reworkedData = []
for i in data:
        reworkedItem = {}
        reworkedItem['userId'] = i['userId']
        reworkedItem['id'] = i['id']
        reworkedItem['gender'] = transform_sex_value(i)
        reworkedItem['sport'] = transform_sport_value(i)
        reworkedItem['distance'] = calculate_tranining_distance(i)
        reworkedItem['duration'] = calculate_training_duration(i)
        reworkedItem['avg'] = calculate_avg(reworkedItem['distance'], reworkedItem['duration'])
        reworkedItem['heart_rate_avg'] = calculate_hear_rate_avg(i)
        reworkedItem['altitude_difference'] = calculate_altitude_difference(i)
        reworkedData.append(reworkedItem)

#### A partire dalla nuova lista creata, si genera un nuovo dataset corrispondente

In [233]:
reworkedCompleteDataSet = pd.DataFrame(reworkedData)

# Elimino le row contenenti dati degli allenamenti non significativi
reworkedCompleteDataSet = reworkedCompleteDataSet[(reworkedCompleteDataSet==0).sum(axis=1)/len(reworkedCompleteDataSet.columns) <= 0.000]

# Per facilitare le operazioni successive, trasformo in stringa tutti i dati contenuti del dataset
reworkedCompleteDataSet = reworkedCompleteDataSet.astype(str)

reworkedCompleteDataSet.head(5)

,userId,id,gender,sport,distance,duration,avg,heart_rate_avg,altitude_difference
0,10921915,396826535,1,1,52,126,24,153,818
1,10921915,392337038,1,1,32,74,25,148,454
2,10921915,389643739,1,1,45,112,24,141,681
3,10921915,386729739,1,1,32,75,25,148,479
4,10921915,383186560,1,1,11,22,30,170,196


#### Funzione che associa ad ogni utente una stringa rappresentante tutti i dati relativi agli allenamenti da lui effettuati.

In [207]:
def getTrainingString(trainingData, usersDictionary):
    oldString = usersDictionary[trainingData['userId']]
    if(oldString != ''):
        newString = oldString +' '+ trainingData['heart_rate_avg']+ '-' +trainingData['gender']+ '-' + trainingData['sport']+ '-' + trainingData['distance']+ '-' + trainingData['duration']+ '-' + trainingData['avg']+ '-' + trainingData['altitude_difference']
    else:
        newString = trainingData['heart_rate_avg']+ '-' +trainingData['gender']+ '-' + trainingData['sport']+ '-' + trainingData['distance']+ '-' + trainingData['duration']+ '-' + trainingData['avg']+ '-' + trainingData['altitude_difference']
    return newString

#### Si crea ora un dictionary contenente la lista degli utenti con associata la stringa corrispondente agli allenamenti fatti.

In [208]:
usersDictionary = {}

for index, row in reworkedCompleteDataSet.iterrows():
    if(row['userId'] not in usersDictionary):
        usersDictionary.update({row['userId']: ''})
        usersDictionary[row['userId']] = getTrainingString(row,usersDictionary)
    else:
        usersDictionary[row['userId']] = getTrainingString(row,usersDictionary)

#### Creo ora un nuvo dataset corrispondente al dictionary sopra citato. Tale dataset conterrà quindi la lista degli utenti con associata la stringa corrispondente ai dati degli allenamenti effettuati. Viene implementata dunque questa trasformazione dei dati per consentire la successiva gestione di tali informazioni tramite TFIDF e la cosine similarity. Si andrà quindi a verificare la "vicinanza" tra gli allenamenti effettuati da un utente con quelli di un altro tramite il tipico approccio utilizzato per i documenti testuali: gli allenamenti di ogni utente vengono difatti interpretati come un singolo documento composto da più frasi (dati dei singoli allenamenti).

In [237]:
usersJson = []

for userId,training_data in usersDictionary.items():
    usersJson.append({
        'userId': userId,
        'training_data': training_data
    })
    
usersTrainingDataSet = pd.DataFrame(usersJson)
usersTrainingDataSet.head()

,userId,training_data
0,10921915,153-1-1-52-126-24-818 148-1-1-32-74-25-454 141...
1,4969375,115-1-2-8-64-7-761 144-1-2-14-70-12-205 128-1-...
2,430859,130-1-1-37-104-21-226 137-1-1-59-168-21-617 13...
3,279317,151-2-2-5-28-10-84 115-2-1-16-47-20-339 129-2-...
4,3905196,141-1-2-14-79-10-713 140-1-2-12-63-11-652 137-...


#### Applico la funzione peso TF-IDF al dataset usersTrainingDataSet ed effettuo il calcolo della cosine similarity

In [240]:
tfidf = TfidfVectorizer(stop_words='english')
usersTrainingDataSet['training_data'] = usersTrainingDataSet['training_data'].fillna('')
overview_matrix = tfidf.fit_transform(usersTrainingDataSet['training_data'])

similarity_matrix = linear_kernel(overview_matrix,overview_matrix)

mapping = pd.Series(usersTrainingDataSet.index,index = usersTrainingDataSet['userId'])

#### Funzione che implementa la ricerca degli utenti con gli allenamenti più simili a quelli dell'utente passato come parametro

In [241]:
def recommend_movies_based_on_plot(movie_input):
    movie_index = mapping[movie_input]
    #get similarity values with other movies
    #similarity_score is the list of index and similarity matrix
    similarity_score = list(enumerate(similarity_matrix[movie_index]))
    #sort in descending order the similarity score of movie inputted with all the other movies
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    # Get the scores of the 10 most similar movies. Ignore the first movie.
    similarity_score = similarity_score[1:10]
    #return movie names using the mapping series
    movie_indices = [i[0] for i in similarity_score]
    return (usersTrainingDataSet['userId'].iloc[movie_indices])

In [243]:
recommend_movies_based_on_plot('11884453')

317      791908
609     2894897
74       260784
842     3411439
263     3276737
417     2587574
625     6000431
189     6113340
1038    3021311
Name: userId, dtype: object

In [230]:
print(usersTrainingDataSet.loc[[usersTrainingDataSet.index[usersTrainingDataSet['userId'] == '11884453'].tolist()[0]]])
print(usersTrainingDataSet.loc[[usersTrainingDataSet.index[usersTrainingDataSet['userId'] == '791908'].tolist()[0]]])
print(usersTrainingDataSet.loc[[usersTrainingDataSet.index[usersTrainingDataSet['userId'] == '2894897'].tolist()[0]]])
print(usersTrainingDataSet.loc[[usersTrainingDataSet.index[usersTrainingDataSet['userId'] == '6000431'].tolist()[0]]])
print(usersTrainingDataSet.loc[[usersTrainingDataSet.index[usersTrainingDataSet['userId'] == '16786'].tolist()[0]]])



       userId                                      training_data
399  11884453  151-1-2-16-82-11-196 132-1-1-127-283-26-1175 1...
     userId                                      training_data
317  791908  132-1-2-23-137-10-1338 138-1-1-48-112-25-957 1...
      userId                                      training_data
609  2894897  139-1-2-10-86-6-411 90-1-2-10-57-10-74 123-1-2...
      userId                                      training_data
625  6000431  138-1-1-64-148-25-1046 148-1-1-86-183-28-1189 ...
   userId                                      training_data
31  16786  138-1-2-39-239-9-531 134-1-1-27-62-26-369 149-...
